<a href="https://colab.research.google.com/github/movva123/AIBasedAutomation/blob/master/Forex_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# ! pip install plotly==4.1.0

In [2]:
!pip install quandl

     |████████████████████████████████| 61kB 4.6MB/s 
     |████████████████████████████████| 61kB 24.6MB/s 
     |████████████████████████████████| 2.3MB 8.3MB/s 
     |████████████████████████████████| 102kB 29.9MB/s 
  Created wheel for inflection: filename=inflection-0.3.1-cp36-none-any.whl size=6076 sha256=a2b8a9f6ec69050703dc8da347c297238c856610ce0ee9483dd6ee8d1202edc9
  Stored in directory: /root/.cache/pip/wheels/9f/5a/d3/6fc3bf6516d2a3eb7e18f9f28b472110b59325f3f258fe9211
Successfully built inflection
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: more-itertools 7.2.0
    Uninstalling more-itertools-7.2.0:
      Successfully uninstalled more-itertools-7.2.0


In [3]:
!pip install nsepy

  Created wheel for nsepy: filename=nsepy-0.7-cp36-none-any.whl size=34388 sha256=a993bb4f8cc52a965a985630fc2deef6b6609f93a5facf18ba1d5e6f1794ff17
  Stored in directory: /root/.cache/pip/wheels/88/89/1e/3dfa14f863314a42481d8926d9ef4825da5d899e026435fc3a
Successfully built nsepy


In [4]:
import pandas as pd
import plotly as py
from plotly import tools
import numpy as np
import plotly.graph_objs as go
import quandl
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from scipy import stats
import warnings
import math
import matplotlib.pyplot as plt
import scipy.optimize
from scipy.optimize import OptimizeWarning
from sklearn.linear_model import LinearRegression
#from matplotlib.finance import _candlestick
from matplotlib.dates import date2num 
from datetime import datetime
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Flatten,Dropout,BatchNormalization
from keras.models import Sequential
from keras.callbacks import EarlyStopping
import seaborn as sns
from nsepy import get_history
from datetime import date
import datetime
from nsepy.derivatives import get_expiry_date
from math import floor,ceil
# Addition of the new Row
import requests
from bs4 import BeautifulSoup
import json



Using TensorFlow backend.


In [0]:
class holder:
  1

  #heikenashi price bars
def heikenashi(prices,periods):
  dict={}
  results=holder()
  HAclose=prices[['open','high','low','close']].sum(axis=1)/4
  HAopen=HAclose.copy()
  HAhigh=HAclose.copy()
  HAlow=HAclose.copy()
  
  HAopen.iloc[0]=HAclose.iloc[0]
  
  for i in range(1,len(prices)):
    HAopen.iloc[i]=(HAopen.iloc[i-1]+HAclose[i-1])/2
    HAhigh.iloc[i]=np.array([prices.high.iloc[i],HAopen.iloc[i],HAclose.iloc[i]]).max()
    HAlow.iloc[i]=np.array([prices.low.iloc[i],HAopen.iloc[i],HAclose.iloc[i]]).min()
  
  df=pd.concat((HAopen,HAclose,HAhigh,HAlow),axis=1)
  df.columns=[['open','high','close','low']]
  df.index=df.index.droplevel(0)
  dict[periods[0]]=df
  results.candles=dict
  return results


# Fourier and Sine series

def detrender(prices,method='difference'):
  if method=='difference':
    detrended= prices.close[1:]-prices.close[:-1].values
  elif method=='linear':
    x=np.arrange(1,len(prices))
    y=prices.close.values
    model=LinearRegression()
    model.fit(x.reshape(-1,1),y.reshape(-1,1))
    trend=model.predict(x.reshape(-1,1))
    trend=trend.reshape((len(prices),))
    detrended=prices.close-trend
  else:
    print('method not called with correct parameters')
  return detrended 
    

def fseries(x,a0,a1,b1,w):
  f= a0+a1*np.cos(w*x)+b1*np.sin(w*x)
  return f

def sseries(x,a0,b1,w):
  s= a0 + b1*np.sin(w*x)
  return s

def fourier(prices,periods,method='difference'):
  results=holder()
  dict={}
  # Use to control whether it needs to be dispalayed or not.
  plot=False
  detrended=detrender(prices,method)
  for i in range(0,len(periods)):
    coeffs=[]
    for j in range(periods[i],len(prices)-periods[i]):
      x=np.arange(0,periods[i])
      y=detrended.iloc[j-periods[i]:j]
      with warnings.catch_warnings():
        warnings.simplefilter('error',OptimizeWarning)
      try:
        res=scipy.optimize.curve_fit(fseries,x,y)
      except (RuntimeError,OptimizeWarning):
        res=np.empty((1,4))
        res[0,:]=np.NAN
      if plot==True:
        xt=np.linspace(0,periods[i],100)
        yt=fseries(xt,res[0][0],res[0][1],res[0][2],res[0][3])
        plt.plot(x,y)
        plt.plot(xt,yt,'r')
        plt.show()
      coeffs=np.append(coeffs,res[0],axis=0)
    warnings.filterwarnings('ignore',category=np.VisibleDeprecationWarning)
    coeffs=np.array(coeffs).reshape(((len(coeffs)//4,4)))
    df=pd.DataFrame(coeffs,index=prices.iloc[periods[i]:-periods[i]].index)
    df.columns=[['a0','a1','b1','w']]
    df=df.fillna(method='bfill')
    dict[periods[i]]=df
  results.coeffs=dict
  return results

def sine(prices,periods,method='difference'):
  results=holder()
  dict={}
  # Use to control whether it needs to be dispalayed or not.
  plot=False
  detrended=detrender(prices,method)
  for i in range(0,len(periods)):
    coeffs=[]
    for j in range(periods[i],len(prices)-periods[i]):
      x=np.arange(0,periods[i])
      y=detrended.iloc[j-periods[i]:j]
      with warnings.catch_warnings():
        warnings.simplefilter('error',OptimizeWarning)
      try:
        res=scipy.optimize.curve_fit(sseries,x,y)
      except (RuntimeError,OptimizeWarning):
        res=np.empty((1,3))
        res[0,:]=np.NAN
      if plot==True:
        xt=np.linspace(0,periods[i],100)
        yt=sseries(xt,res[0][0],res[0][1],res[0][2])
        plt.plot(x,y)
        plt.plot(xt,yt,'r')
        plt.show()
      coeffs=np.append(coeffs,res[0],axis=0)
    warnings.filterwarnings('ignore',category=np.VisibleDeprecationWarning)
    coeffs=np.array(coeffs).reshape(((len(coeffs)//3,3)))
    df=pd.DataFrame(coeffs,index=prices.iloc[periods[i]:-periods[i]].index)
    df.columns=[['a0','b1','w']]
    df=df.fillna(method='bfill')
    dict[periods[i]]=df
    
  results.coeffs=dict
  return results

# WADL price series

def wadl(prices,periods):
  results=holder()
  dict={}
  
  for i in range(0, len(periods)):
    WAD=[]
    for j in range(periods[i],len(prices)-periods[i]):
      TRH=np.array([prices.high.iloc[j],prices.close.iloc[j-1]]).max()
      TRL=np.array([prices.low.iloc[j],prices.close.iloc[j-1]]).min()
      
      if prices.close.iloc[j]> prices.close.iloc[j-1]:
        PM=prices.close.iloc[j]-TRL
      elif prices.close.iloc[j]<prices.close.iloc[j-1]:
        PM=prices.close.iloc[j]-TRH
      elif prices.close.iloc[j]==prices.close.iloc[j-1]:
        PM=0
      else:
        print('error in Wadl function')
      AD=PM*prices.volume.iloc[j]
      WAD=np.append(WAD,AD)
    WAD=WAD.cumsum()
    WAD=pd.DataFrame(WAD,index=prices.iloc[periods[i]:-periods[i]].index)
    WAD.columns=[['close']]
    dict[periods[i]]=WAD
  results.wadl=dict
  return results

def OHLCresampleDataFrame(DataFrame,TimeFrame,column='close'):
  grouped=DataFrame.groupby('Symbol')
  if np.any(DataFrame.columns=='close'):
    open=grouped['open'].resample(TimeFrame).ohlc()
    close=grouped['close'].resample(TimeFrame).ohlc()
    high=grouped['high'].resample(TimeFrame).ohlc()
    low=grouped['low'].resample(TimeFrame).ohlc()
    volume=grouped['volume'].resample(TimeFrame).count()
    
    resampled=pd.DataFrame(open)
    resampled['close']=close
    resampled['high']=high
    resampled['low']=low
    resampled['volume']=volume
    
  resampled=resampled.dropna()
  return resampled

#Momentum Series

def momentum(prices,periods):
  results=holder()
  open={}
  close={}
  for i in range(0,len(periods)):
    open[periods[i]]=pd.DataFrame(prices.open.iloc[periods[i]:]-prices.open.iloc[:-periods[i]].values,
                                  index=prices.iloc[periods[i]:].index)
    close[periods[i]]=pd.DataFrame(prices.close.iloc[periods[i]:]-prices.close.iloc[:-periods[i]].values,
                                  index=prices.iloc[periods[i]:].index)
    open[periods[i]].columns=[['open']]
    close[periods[i]].colums=[['close']]
  results.open=open
  results.close=close
  return results


# Stochastic Series

def stochastic(prices,periods):
  results=holder()
  close={}
  for i in range(0,len(periods)):
    Ks=[]
    for j in range(periods[i],len(prices)-periods[i]):
      C=prices.close.iloc[j+1]
      H=prices.high.iloc[j-periods[i]:j].max()
      L=prices.low.iloc[j-periods[i]:j].min()
      
      if H==L:
        K=0
      else:
        K=100*(C-L)/(H-L)
      Ks=np.append(Ks,K)
    df_temp=pd.DataFrame(Ks,index=prices.iloc[periods[i]+1:-periods[i]+1].index)
    df_temp.columns=[['K']]
    df_temp['D']=df_temp.K.rolling(3).mean()
    df_temp=df_temp.dropna() 
    close[periods[i]]=df_temp
  results.close=close 
  return results

# ADOSC series 

def adosc(prices,periods):
  results=holder()
  accdist={}
  for i in range(0,len(periods)):
    AD=[]
    for j in range(periods[i],len(prices)-periods[i]):
      C=prices.close.iloc[j+1]
      H=prices.high.iloc[j-periods[i]:j].max()
      L=prices.low.iloc[j-periods[i]:j].min()
      V=prices.volume.iloc[j+1]
      if H==L:
        CLV=0
      else:
        CLV=((C-L)-(H-C))/(H-L)
      AD=np.append(AD,CLV*V)
    AD=AD.cumsum()
    AD=pd.DataFrame(AD,index=prices.iloc[periods[i]+1:-periods[i]+1].index)
    AD.columns=[['AD']]
    accdist[periods[i]]=AD
  results.AD=accdist
  return results

#PROC series 

def proc(prices,periods):
  results=holder()
  proc={}
  for i in range(0,len(periods)):
    proc[periods[i]]=pd.DataFrame((prices.close.iloc[periods[i]:]-prices.close.iloc[:-periods[i]].values)/prices.close.iloc[:-periods[i]].values)
    proc[periods[i]].columns=[['close']]
  results.proc=proc
  return results

# Williams Series 

def williams(prices,periods):
  results=holder()
  close={}
  for i in range(0,len(periods)):
    RS=[]
    for j in range(periods[i],len(prices)-periods[i]):
      C=prices.close.iloc[j+1]
      H=prices.high.iloc[j-periods[i]:j].max()
      L=prices.low.iloc[j-periods[i]:j].min()
      if H==L:
        R=0
      else:
        R=-100*(H-C)/(H-L)
      RS=np.append(RS,R)
    df=pd.DataFrame(RS,index=prices.iloc[periods[i]+1:-periods[i]+1].index)
    df.columns=[['R']]
    df=df.dropna()
    close[periods[i]]=df
  results.close=close
  return results

# MACD 

def MACD(prices,periods):
  results=holder()
  EMA1=prices.close.ewm(span=periods[0]).mean()
  EMA2=prices.close.ewm(span=periods[1]).mean()
  MACD=pd.DataFrame(EMA1-EMA2)
  MACD.columns=[['L']]
  
  SigMACD=MACD.rolling(3).mean()
  SigMACD.columns=[['SL']]
  
  results.line=MACD
  results.signal=SigMACD
  
  return results

# CCI series

def cci(prices,periods):
  results=holder()
  CCI={}
  for i in range(0, len(periods)):
    MA=prices.close.rolling(periods[i]).mean()
    STD=prices.close.rolling(periods[i]).std()
    D=(prices.close-MA)/STD
    CCI[periods[i]]=pd.DataFrame((prices.close-MA)/(D*0.015))
    CCI[periods[i]].columns=[['close']]
  results.cci=CCI
  return results


# Bollinger Bands

def bollinger(prices,periods,deviation):
  results=holder()
  boll={}
  
  for i in range(0,len(periods)):
    mid=prices.close.rolling(periods[i]).mean()
    std=prices.close.rolling(periods[i]).std()
    
    upper=mid+deviation*std
    lower=mid+deviation*std
    
    df=pd.concat((upper,mid,lower),axis=1)
    df.columns=[['upper','mid','lower']]
    boll[periods[i]]=df
  results.bands=boll
  return results


# Averages

def paverage(prices,periods):
  results=holder()
  avg={}
  
  for i in range(0,len(periods)):
    avg[periods[i]]=pd.DataFrame(prices[['open','low','close','high']].rolling(periods[i]).mean())
  results.avg=avg
  return results

# Slope

def slope(prices,periods):
  results=holder()
  slope={}
  for i in range(0,len(periods)):
    ms=[]
    for j in range(periods[i],len(prices)-periods[i]):
      y=prices.high.iloc[j-periods[i]:j].values
      x=np.arange(0,len(y))
      
      res=stats.linregress(x,y=y)
      m=res.slope
      ms=np.append(ms,m)
    ms=pd.DataFrame(ms,index=prices.iloc[periods[i]:-periods[i]].index)
    ms.columns=[['high']]
    slope[periods[i]]=ms
  results.slope=slope
  return results



In [0]:
# Current row for Nifty()

def current_days_data():
  base_url = 'https://beta.nseindia.com'
  r = requests.get(base_url)
  soup = BeautifulSoup(r.text, 'html.parser')
  all_scripts = soup.find_all('script')
  data = json.loads(all_scripts[1].get_text()[28:])
  print(data['indexDataInfo'])
  symbol=data['indexDataInfo'][0]['name']
  stock_dict=data['indexDataInfo'][0]['metadata']
  current_datetime=datetime.datetime.strptime(stock_dict['timeVal'], '%d-%b-%Y %H:%M:%S')
  current_date=datetime.datetime.strftime(current_datetime,'%d-%b-%Y')
  current_hours=int(datetime.datetime.strftime(current_datetime,'%H'))
  current_mins=int(datetime.datetime.strftime(current_datetime,'%M'))
  stock_data=[stock_dict['open'],stock_dict['high'],stock_dict['last'],stock_dict['low'],stock_dict['totalTradedVolume']]
  return stock_data,current_datetime,current_hours,current_mins

In [0]:
def create_masterframe(prices):
  momentumKey=[3,4,5,8,9,10]
  stochasticKey=[3,4,5,8,9,10]
  williamsKey=[6,7,8,9,10]
  procKey=[12,13,14,15]
  wadlKey=[15]
  adoscKey=[2,3,4,5]
  macdKey=[15,30]
  cciKey=[15]
  bollingerKey=[15]

  heikenashiKey=[15]
  paverageKey=[2]
  slopeKey=[3,4,5,10,20,30]
  fourierKey=[10,20,30]
  sineKey=[5,6]

  keyList=[momentumKey,stochasticKey,williamsKey,procKey,wadlKey,adoscKey,macdKey,cciKey,bollingerKey,heikenashiKey,paverageKey,slopeKey,fourierKey,sineKey]

  momentumDict=momentum(prices,momentumKey)
  print(1)
  stochasticDict=stochastic(prices,stochasticKey)
  print(2)
  williamsDict=williams(prices,williamsKey)
  print(3)
  procDict=proc(prices,procKey)
  print(4)
  wadlDict=wadl(prices,wadlKey)
  print(6)
  adoscDict=adosc(prices,adoscKey)
  print(6)
  macdDict=MACD(prices,macdKey)
  print(7)
  cciDict=cci(prices,cciKey)
  print(8)
  bollingerDict=bollinger(prices,bollingerKey,2)
  print(9)
  hkaprices=prices.copy()
  hkaprices['Symbol']='SYMB'
  resampled=OHLCresampleDataFrame(hkaprices,'15D')
  heikenDict=heikenashi(resampled,heikenashiKey)
  print(10)
  paverageDict=paverage(prices,paverageKey)
  print(11)
  slopeDict=slope(prices,slopeKey)
  print(12)
  fourierDict=fourier(prices,fourierKey)
  print(13)
  sineDict=sine(prices,sineKey)
  print(14)


  dictList=[momentumDict.close,stochasticDict.close,williamsDict.close,procDict.proc,wadlDict.wadl,adoscDict.AD,macdDict.line,cciDict.cci,bollingerDict.bands,heikenDict.candles,paverageDict.avg,
            slopeDict.slope,fourierDict.coeffs,sineDict.coeffs]

  colFeat=['momemntum','stochastic','proc','will','wadl','adosc','macd','cci','bollinger','heiken','paverage','slope','fourier','sine']

  #populate masterframe
  masterFrame=pd.DataFrame(index=prices.index)

  for i in range(0,len(dictList)):
    if colFeat[i]=='macd':
      colID=colFeat[i]+str(keyList[6][0]+keyList[6][1])
      masterFrame[colID]=dictList[i]
    else:
      for j in keyList[i]:
  #       print(dictList[i][j].columns)
        for k in dictList[i][j].columns.get_level_values(0):
  #         print('parameters:',i,j,k)
          colID=colFeat[i]+str(j)+k
          masterFrame[colID]=dictList[i][j][k]
  threshold=round(0.7*len(masterFrame))
  masterFrame[['open','high','close','low','volume']]= prices[['open','high','close','low','volume']]
  # masterFrame['shift_high']=masterFrame['high'].shift(1)
  masterFrame['higher_1']=masterFrame['high']<masterFrame['high'].shift(1)
  masterFrame['higher_1']=masterFrame['higher_1'].astype(int)
#   masterFrame.hiken15open=masterFrame.heiken15open.fillna(method='bfill')
#   masterFrame.hiken15high=masterFrame.heiken15high.fillna(method='bfill')      
#   masterFrame.hiken15low=masterFrame.heiken15low.fillna(method='bfill')        
#   masterFrame.hiken15close=masterFrame.heiken15close.fillna(method='bfill')
  masterFrame.fillna(method='ffill',inplace=True)
  masterFrameCleaned=masterFrame.copy()
  masterFrameCleaned=masterFrameCleaned.dropna(axis=1,thresh=threshold)
  masterFrameCleaned=masterFrameCleaned.dropna(axis=0)
  masterFrame.to_csv('masterFrame.csv')
  masterFrame.to_csv('masterFrameCleaned.csv')
  return masterFrameCleaned

In [8]:
# Data Load Functions
masterFrameCleaned=pd.DataFrame()
DataLoad=[['WIKI/PRICES','GE','2000-12-31','2014-12-31'],['WIKI/PRICES','JPM','2000-12-31','2014-12-31'],['WIKI/PRICES','GS','2000-12-31','2014-12-31']]

for i in DataLoad:
  (table,instr,start_date,end_date)=i
  quandl.ApiConfig.api_key = 'btemzC5zy7Sy4ErTGqKs'
  dt = quandl.get_table(table, ticker=[instr],qopts={'columns': ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']},date={'gte': start_date, 'lte': end_date},paginate=True)
  df = pd.DataFrame(dt)
  df = df.sort_values(by='date', ascending=True)
  
  df.date=pd.to_datetime(df.date,format='%Y-%m-%d')
  df = df.sort_values(by='date', ascending=True)
  df=df.set_index(df.date)
  df=df[['open','high','close','low','volume']]
  df.to_csv('raw_data.csv')
  high_price_value=df["high"].max()
  high_volume_value=df["volume"].max()
  for i in df.columns:
    if i !='volume':
      df[i]=df[i].div(high_price_value)
    else:
      df[i]=df[i].div(high_volume_value)
  prices=df
  masterFrame=create_masterframe(prices)
  masterFrameCleaned=pd.concat([masterFrameCleaned,masterFrame])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:182: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access



1
2
3
4
6
6
7
8
9
10
11
12
13
14


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:182: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access



1
2
3
4
6
6
7
8
9
10
11
12
13
14


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:182: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access



1
2
3
4
6
6
7
8
9
10
11
12
13
14


In [9]:
masterFrameCleaned.shape

(10473, 75)

To do : Address the problem with looping for multiple price datas

In [0]:
scaler=MinMaxScaler()
y=masterFrameCleaned['higher_1'].values
x=masterFrameCleaned.drop(columns='higher_1',axis=1).values
y=masterFrameCleaned['higher_1'].values
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=22)
col_nums=x_train.shape[1]

In [11]:
masterFrameCleaned.shape

(10473, 75)

In [12]:
model=Sequential()
model.add(Dense(64,input_shape=(col_nums,),activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Dense(1,activation='sigmoid'))


model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
earlystopping=EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=False)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


#To do items 
### 1.) Optimize the layers of the model 
### 2.) Optimize hyperparameters
### 3.) Optimize the parameters for the trading features
### 4.) Identify the best way to identify the high of the day.


In [13]:
model.fit(x_train,y_train,validation_split=0.2,epochs=20,callbacks=[earlystopping])

Train on 6702 samples, validate on 1676 samples
Epoch 1/20
6702/6702 [==============================] - 8s 1ms/step - loss: 0.6463 - acc: 0.6861 - val_loss: 0.5113 - val_acc: 0.7482
Epoch 2/20
6702/6702 [==============================] - 3s 389us/step - loss: 0.5698 - acc: 0.7183 - val_loss: 0.4901 - val_acc: 0.7721
Epoch 3/20
6702/6702 [==============================] - 3s 391us/step - loss: 0.5408 - acc: 0.7366 - val_loss: 0.5034 - val_acc: 0.7595


In [14]:
model.evaluate(x_test,y_test)

2095/2095 [==============================] - 0s 68us/step


[0.4797896719206489, 0.7642004774123217]

In [15]:
data = get_history(symbol="NIFTY 50", start=date(2001,1,1), end=date(2014,12,31),index='True')
data.columns = map(str.lower, data.columns)
data.index.names=['date']
data.head()
data=data[['open','high','close','low','volume']]
data = data.sort_values(by='date', ascending=True)
stock_data,current_datetime,current_hours,current_mins=current_days_data()
data.index=pd.to_datetime(data.index,format='%Y-%m-%d')
# df_temp={'date':current_datetime,'open':stock_data[0],'high':stock_data[1],'close':stock_data[2],'low':stock_data[3],'volume':stock_data[4]}
data.loc[current_datetime]=stock_data
data.to_csv('raw_data.csv')
high_price_value=data["high"].max()
high_volume_value=data["volume"].max()
for i in data.columns:
  if i !='volume':
    data[i]=data[i].div(high_price_value)
  else:
    data[i]=data[i].div(high_volume_value)
  prices=data
masterFrameCleaned=create_masterframe(prices)
  

[{'name': 'NIFTY 50', 'indexName': 'NIFTY 50', 'metadata': {'indexName': 'NIFTY 50', 'open': 11542.7, 'high': 11694.85, 'low': 11471.35, 'previousClose': 11274.2, 'last': 11600.2, 'percChange': 2.92, 'change': 326, 'timeVal': '23-Sep-2019 15:32:25', 'yearHigh': 12103.05, 'yearLow': 10004.55, 'totalTradedValue': 2814785.51, 'totalTradedVolume': 741546687}, 'advance': {'declines': '18', 'advances': '32', 'unchanged': '0'}, 'topGainers': {'data': [{'symbol': 'BPCL', 'identifier': 'BPCLEQN', 'lastPrice': 459, 'pChange': 13.6560604184722, 'totalTradedVolume': 31006317, 'totalTradedValue': 134610.82}, {'symbol': 'LT', 'identifier': 'LTEQN', 'lastPrice': 1540.45, 'pChange': 9.13567127169678, 'totalTradedVolume': 12082820, 'totalTradedValue': 184420.08}, {'symbol': 'BAJFINANCE', 'identifier': 'BAJFINANCEEQN', 'lastPrice': 4032, 'pChange': 8.908216735994818, 'totalTradedVolume': 4011002, 'totalTradedValue': 157976.12}, {'symbol': 'EICHERMOT', 'identifier': 'EICHERMOTEQN', 'lastPrice': 19450, 'p

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:182: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access



2
3
4
6
6
7
8
9
10
11
12
13
14


In [16]:
masterFrameCleaned.shape

(3461, 75)

In [0]:
scaler=MinMaxScaler()
y=masterFrameCleaned['higher_1'].values
x=masterFrameCleaned.drop(columns='higher_1',axis=1).values
y=masterFrameCleaned['higher_1'].values
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=22)
col_nums=x_train.shape[1]

In [18]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=20,callbacks=[earlystopping])

Train on 2768 samples, validate on 693 samples
Epoch 1/20
2768/2768 [==============================] - 1s 380us/step - loss: 0.4972 - acc: 0.7616 - val_loss: 0.4120 - val_acc: 0.8326
Epoch 2/20
2768/2768 [==============================] - 1s 365us/step - loss: 0.4637 - acc: 0.7919 - val_loss: 0.4239 - val_acc: 0.8225


In [19]:
data = get_history(symbol="NIFTY 50", start=date(2018,8,1), end=date(2019,9,30),index='True')
data.columns = map(str.lower, data.columns)
data.index.names=['date']
data.head()
data=data[['open','high','close','low','volume']]
data = data.sort_values(by='date', ascending=True)
data.index=pd.to_datetime(data.index,format='%Y-%m-%d')
stock_data,current_datetime,current_hours,current_mins=current_days_data()
data.loc[current_datetime]=stock_data
data.to_csv('raw_data.csv')
high_price_value=data["high"].max()
high_volume_value=data["volume"].max()
for i in data.columns:
  if i !='volume':
    data[i]=data[i].div(high_price_value)
  else:
    data[i]=data[i].div(high_volume_value)
  prices=data
masterFrameCleaned=create_masterframe(prices)

[{'name': 'NIFTY 50', 'indexName': 'NIFTY 50', 'metadata': {'indexName': 'NIFTY 50', 'open': 11542.7, 'high': 11694.85, 'low': 11471.35, 'previousClose': 11274.2, 'last': 11600.2, 'percChange': 2.92, 'change': 326, 'timeVal': '23-Sep-2019 15:32:25', 'yearHigh': 12103.05, 'yearLow': 10004.55, 'totalTradedValue': 2814785.51, 'totalTradedVolume': 741546687}, 'advance': {'declines': '18', 'advances': '32', 'unchanged': '0'}, 'topGainers': {'data': [{'symbol': 'BPCL', 'identifier': 'BPCLEQN', 'lastPrice': 459, 'pChange': 13.6560604184722, 'totalTradedVolume': 31006317, 'totalTradedValue': 134610.82}, {'symbol': 'LT', 'identifier': 'LTEQN', 'lastPrice': 1540.45, 'pChange': 9.13567127169678, 'totalTradedVolume': 12082820, 'totalTradedValue': 184420.08}, {'symbol': 'BAJFINANCE', 'identifier': 'BAJFINANCEEQN', 'lastPrice': 4032, 'pChange': 8.908216735994818, 'totalTradedVolume': 4011002, 'totalTradedValue': 157976.12}, {'symbol': 'EICHERMOT', 'identifier': 'EICHERMOTEQN', 'lastPrice': 19450, 'p

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:182: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access



2
3
4
6
6
7
8
9
10
11
12
13
14


In [20]:
print(current_datetime)

df_temp=masterFrameCleaned.drop(columns='higher_1',axis=1)
x_temp=np.array(df_temp.loc[current_datetime])
x_predict=np.expand_dims(x_temp,axis=0)
print(x_predict)
x_predict.shape
y=model.predict(x_predict)
labels = (y < 0.5).astype(np.int)
print(y,labels)

2019-09-23 15:32:25
[[ 7.39813518e-02  6.27569084e-02  6.46613870e-02  5.10119350e-02
   4.66411359e-02  4.93388030e-02  1.86455894e+02  2.62783444e+01
  -3.18458066e+01 -4.82376925e+01 -2.52504495e+01  7.44407772e+00
   1.06840655e+02  8.05692290e+01  5.98002023e+01  6.84973810e+01
   7.89833532e+01  7.09142555e+01 -9.94039858e+01 -2.33303518e+01
   6.84065489e+00 -4.01997977e+01 -2.10166468e+01  6.93498281e-02
   6.96702983e-02  7.43014845e-02  5.23393736e-02 -3.92145834e-02
   5.15071549e-01 -4.05130642e+00 -5.87606290e+00 -4.02193371e+00
   1.84571375e-03  1.49438289e+00  9.55760274e-01  9.10928788e-01
   9.55760274e-01  9.29558222e-01  9.06486382e-01  9.29558222e-01
   8.88747051e-01  9.53701753e-01  9.47806545e-01  9.58452621e-01
   9.66272964e-01 -6.92180897e-03 -2.28620059e-03  6.39095104e-04
  -1.04198629e-03 -1.31078623e-03 -2.78775279e-03  1.87321396e-03
   9.37610739e-03  9.34500676e-04  6.15270537e-01 -1.92631530e-03
   4.89806681e-03 -4.01051831e-05  1.28327324e+00 -2.770

In [21]:
masterFrameCleaned

,momemntum3close,momemntum4close,momemntum5close,momemntum8close,momemntum9close,momemntum10close,stochastic3K,stochastic3D,stochastic4K,stochastic4D,stochastic5K,stochastic5D,stochastic8K,stochastic8D,stochastic9K,stochastic9D,stochastic10K,stochastic10D,proc6R,proc7R,proc8R,proc9R,proc10R,will12close,will13close,will14close,will15close,wadl15close,adosc2AD,adosc3AD,adosc4AD,adosc5AD,macd45,cci15close,bollinger15upper,bollinger15mid,bollinger15lower,heiken15open,heiken15high,heiken15close,heiken15low,paverage2open,paverage2low,paverage2close,paverage2high,slope3high,slope4high,slope5high,slope10high,slope20high,slope30high,fourier10a0,fourier10a1,fourier10b1,fourier10w,fourier20a0,fourier20a1,fourier20b1,fourier20w,fourier30a0,fourier30a1,fourier30b1,fourier30w,sine5a0,sine5b1,sine5w,sine6a0,sine6b1,sine6w,open,high,close,low,volume,higher_1
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-09-17 00:00:00,0.007457,-0.004986,-0.017463,-0.011778,-0.016905,-0.025014,39.513631,26.719126,36.153628,32.671862,36.153628,32.671862,25.428628,23.073041,25.428628,23.081730,25.428628,23.075018,-63.846372,-63.846372,-74.571372,-74.571372,-74.571372,-0.026869,-0.030732,-0.026873,-0.015519,-0.003724,0.959317,0.876710,0.641055,0.796917,-0.001138,0.753887,0.976501,0.953885,0.976501,0.939932,0.965131,0.968983,0.939932,0.946392,0.941806,0.945751,0.949686,0.001812,-0.002048,-0.002906,-0.002464,-0.000264,0.000599,-0.002095,-0.003299,-0.010534,1.407698,-0.000450,0.003855,0.002466,1.081478,0.000044,-0.003305,-0.000114,0.995873,-0.017934,0.024386,0.665467,-0.001588,-0.014463,1.338235,0.947278,0.947278,0.940073,0.939176,0.153026,1
2018-09-18 00:00:00,-0.007519,-0.000711,-0.013154,-0.016364,-0.019945,-0.025072,10.510895,41.112532,8.890954,39.452551,8.134921,39.200540,8.134921,28.014974,5.721691,27.210564,5.721691,27.161403,-91.865079,-91.865079,-91.865079,-94.278309,-94.278309,-0.034076,-0.035324,-0.039153,-0.035328,-0.005728,0.814781,0.732174,0.490590,0.643684,-0.001966,0.806349,0.975800,0.951609,0.975800,0.939932,0.965131,0.968983,0.939932,0.943832,0.935130,0.935989,0.945068,0.003478,0.000819,-0.001423,-0.002116,-0.000615,0.000474,-0.000921,-0.008383,-0.007378,1.559294,-0.000924,0.004589,-0.000344,1.116665,0.000182,-0.000575,-0.003120,1.110016,-0.006082,0.013435,0.974890,-0.015476,0.020224,0.594681,0.940387,0.942857,0.931906,0.931083,0.183008,1
2018-09-19 00:00:00,-0.023205,-0.011200,-0.004391,-0.024998,-0.020045,-0.023626,-5.804798,14.739910,-5.804798,13.079928,-4.910161,13.126129,-4.492630,9.690306,-4.492630,8.885896,-3.159888,9.330144,-104.492630,-104.492630,-104.492630,-104.492630,-103.159888,-0.038196,-0.037891,-0.039134,-0.042948,-0.007266,0.301357,0.519590,0.278007,0.434509,-0.002824,0.825295,0.973591,0.948832,0.973591,0.939932,0.965131,0.968983,0.939932,0.938119,0.928685,0.930065,0.939577,-0.004619,0.000279,-0.000427,-0.001612,-0.000984,0.000282,-0.002296,-0.002700,0.011307,1.437735,-0.001032,0.001556,-0.003852,1.118050,-0.000341,-0.001912,0.003463,1.049292,0.011212,-0.018706,0.525321,-0.002717,0.011167,1.236151,0.935851,0.936297,0.928225,0.926287,0.190471,1
2018-09-21 00:00:00,-0.019388,-0.030744,-0.018739,-0.036850,-0.032537,-0.027584,-49.488793,-14.927565,-39.223585,-12.045810,-39.223585,-11.999609,-30.357143,-8.904951,-30.357143,-9.709361,-30.357143,-9.265114,-133.178439,-130.357143,-130.357143,-130.357143,-130.357143,-0.037924,-0.046008,-0.045706,-0.046938,-0.016460,-0.946410,-0.567353,-0.696786,-0.540284,-0.003911,0.891756,0.972562,0.945810,0.972562,0.939932,0.965131,0.968983,0.939932,0.933564,0.912057,0.924455,0.936906,-0.005490,-0.005181,-0.001728,-0.001947,-0.001334,0.000067,-0.004082,0.005780,0.008477,1.636115,-0.001527,-0.003976,0.001745,1.016550,-0.000670,0.001560,0.004010,1.051466,0.007409,-0.017895,0.612883,-0.002511,0.008447,1.985525,0.931278,0.937516,0.920685,0.897827,0.546264,0
2018-09-24 00:00:00,-0.025737,-0.033905,-0.045261,-0.038891,-0.051367,-0.047054,-95.847274,-50.380288,-77.957420,-4

In [0]:
from datetime import date
def nse_get_future_prices(stock,transaction_date,duration):
  expiry=get_expiry_date(year=transaction_date.year,month=transaction_date.month)
  start_date=date(transaction_date.year,transaction_date.month,transaction_date.day)
  if start_date>expiry and transaction_date.month<12:
    expiry=get_expiry_date(year=transaction_date.year, month=transaction_date.month+1)
  elif start_date>expiry and transaction_date.month==12:
    expiry=get_expiry_date(year=transaction_date.year+1, month=1)
  end_datetime=transaction_date + datetime.timedelta(days=duration)
  end_date=date(end_datetime.year,end_datetime.month,end_datetime.day)
  data=get_history(symbol=stock,start=start_date,end=end_date,expiry_date=expiry,futures=True,index=True)
  data.columns = map(str.lower, data.columns)
  data.index.names=['date']
  data=data[['open','high','close','low','last']]
  return data 

In [0]:
# For Futures

position=[]
transactions=[]
profit=0
total_profit=0
from datetime import date
df=masterFrameCleaned.drop(columns='higher_1',axis=1)
for row in df.itertuples(index=True, name='Pandas'):
  transaction_date=row[0]
  print('Transaction Date:',transaction_date)
  future_data=nse_get_future_prices("NIFTY",transaction_date,0)
  if future_data.shape[0]!=1:
    print('Unique value voilation')
    print(future_data)
    positions=[]
    profit=0 
    continue
  x=np.asarray(row[1:])
  x_predict=np.asarray([x])
  x_predict.shape
  y=model.predict(x_predict)
  labels = (y <0.5).astype(np.int)

  if len(position) > 0:
    
    if future_data['open'][0]-position[0]+25 < future_data['high'][0]:
      profit=(future_data['open'][0]-position[0]+25)
    else:
      profit=(future_data['close'][0]-position[0])
    total_profit += profit
    transactions.append(profit)
    position=[]
    print(profit)
    profit=0

  if labels[0][0]==1:
    position=[future_data["close"][0]]
    
print("total profit",total_profit)
    
  

Transaction Date: 2018-09-17 00:00:00
building dictionary
Transaction Date: 2018-09-18 00:00:00
Transaction Date: 2018-09-19 00:00:00
Transaction Date: 2018-09-21 00:00:00
Transaction Date: 2018-09-24 00:00:00
Transaction Date: 2018-09-25 00:00:00
Transaction Date: 2018-09-26 00:00:00
73.39999999999964
Transaction Date: 2018-09-27 00:00:00
36.04999999999927
Transaction Date: 2018-09-28 00:00:00
Transaction Date: 2018-10-01 00:00:00
Transaction Date: 2018-10-03 00:00:00
Transaction Date: 2018-10-04 00:00:00
Transaction Date: 2018-10-05 00:00:00
Transaction Date: 2018-10-08 00:00:00
Transaction Date: 2018-10-09 00:00:00
Transaction Date: 2018-10-10 00:00:00
Transaction Date: 2018-10-11 00:00:00
-232.0
Transaction Date: 2018-10-12 00:00:00
Transaction Date: 2018-10-15 00:00:00
39.0
Transaction Date: 2018-10-16 00:00:00
38.599999999998545
Transaction Date: 2018-10-17 00:00:00
125.14999999999964
Transaction Date: 2018-10-19 00:00:00
Transaction Date: 2018-10-22 00:00:00
Transaction Date: 20

In [0]:
def get_nse_option_prices(stock,option_type,strike_price,transaction_date,duration):
#   datee = datetime.datetime.strptime(transaction_date, "%Y-%m-%d")
#   month=datee.month
#   year=datee.year
  expiry = get_expiry_date(year=transaction_date.year, month=transaction_date.month)
  start_date=date(transaction_date.year,transaction_date.month,transaction_date.day)
  if start_date>expiry and transaction_date.month<12:
    expiry=get_expiry_date(year=transaction_date.year, month=transaction_date.month+1)
  elif start_date>expiry and transaction_date.month==12:
    expiry=get_expiry_date(year=transaction_date.year+1, month=1)
  end_datetime=transaction_date + datetime.timedelta(days=duration)
  end_date=date(end_datetime.year,end_datetime.month,end_datetime.day)
  print(start_date,end_date,expiry)
  data = get_history(symbol=stock,
                        start=start_date,
                        end=start_date,
                        option_type=option_type,
                        strike_price=strike_price,
                        expiry_date=expiry,
                        index=True)
  data.columns = map(str.lower, data.columns)
  data.index.names=['date']
  # data.rename(columns={'number of contracts':'volume'},inplace=True)
  data=data[['open','high','close','low','last']]
  return data


In [0]:
positions=[]
transactions=[]
diff=[]
profit=0
total_profit=0
df=masterFrameCleaned.drop(columns='higher_1',axis=1)
for row in df.itertuples(index=True, name='Pandas'): 
  print('-'*50)
  transaction_date=row[0]
  print('Transaction Date:',transaction_date)
  x=np.asarray(row[1:])
  x_predict=np.asarray([x])
  x_predict.shape
  y=model.predict(x_predict)
  labels = (y > 0.5).astype(np.int)
  if len(positions)>0:
    print('-'*50)
    print('Settlement of previous positions')
    strike_price=positions[0]
    print('Strike Price:',positions[0])
    print('Buy Price:',positions[1])
    stock_options=get_nse_option_prices("NIFTY","CE",strike_price,transaction_date,0)
    print('Open Price:',stock_options["open"][0])
    print('High Price:',stock_options["high"][0])
    if stock_options.shape[0]!= 1:
      print(stock_options)
      print("unique values have not been recieved")
    else:
      if stock_options["high"][0]>stock_options["open"][0]+25:
        profit=stock_options["open"][0] - positions[1]+25
#       if stock_options["high"][0]>positions[2]:
#         profit=positions[2] - positions[1]
      else:
        profit=stock_options["close"][0] - positions[1]
#       profit= stock_options["open"][0] - stock_options["last"][0]
      print('Profit:',profit)
      diff.append(stock_options["high"][0]-stock_options["open"][0])
      total_profit+=profit
      profit=0
      positions=[]
  if labels[0][0]==1:
    high=row[-4]
    strike_price=floor((high*high_price_value)/50)*50
    stock_options=get_nse_option_prices("NIFTY","CE",strike_price,transaction_date,0)
    if stock_options.shape[0] > 0:
      buy=stock_options["close"][0]
      previous_high=stock_options["high"][0]
      positions.append(strike_price)
      positions.append(buy)
      positions.append(previous_high)
    
      
print('total profits',total_profit)      
 
  

In [0]:
print(total_profit)

In [0]:
positions=[]
transactions=[]
diff=[]
profit=0
total_profit=0
df=masterFrameCleaned.drop(columns='higher_1',axis=1)
for row in df.itertuples(index=True, name='Pandas'): 
  print('-'*50)
  transaction_date=row[0]
  print('Transaction Date:',transaction_date)
  x=np.asarray(row[1:])
  x_predict=np.asarray([x])
  x_predict.shape
  y=model.predict(x_predict)
  labels = (y < 0.5).astype(np.int)
  if len(positions)>0:
    print('-'*50)
    print('Settlement of previous positions')
    strike_price=positions[0]
    print('Strike Price:',positions[0])
    print('Buy Price:',positions[1])
    stock_options=get_nse_option_prices("NIFTY","CE",strike_price,transaction_date,0)
    print('Open Price:',stock_options["open"][0])
    print('High Price:',stock_options["high"][0])
    if labels[0][0]!=1:
      if stock_options.shape[0]!= 1:
        print(stock_options)
        print("unique values have not been recieved")
      else:
          profit=stock_options["close"][0] - positions[1]
#       profit= stock_options["open"][0] - stock_options["last"][0]
      print('Profit:',profit)
      diff.append(stock_options["high"][0]-stock_options["open"][0])
      total_profit+=profit
      transactions.append([transaction_date,positions[1],stock_options["close"][0],profit])
      profit=0
      positions=[]
  if labels[0][0]==1 and len(positions)==0:
    high=row[-4]
    strike_price=floor((high*high_price_value)/50)*50
    stock_options=get_nse_option_prices("NIFTY","CE",strike_price,transaction_date,0)
    if stock_options.shape[0] > 0:
      buy=stock_options["close"][0]
      previous_high=stock_options["high"][0]
      positions.append(strike_price)
      positions.append(buy)
      positions.append(previous_high)
    
      
print('total profits',total_profit)

In [0]:
transactions=pd.DataFrame(transactions)

In [0]:
transactions.columns= ["date","buy","sell","profit"]

In [0]:
masterFrameCleaned.head()